# Databricks Training: Import i eksploracja danych

**Moduł:** Fundamentals & Data Exploration  
**Temat:** DataFrame Reader API, opcje wczytywania, konstrukcja schematów, podstawowe operacje eksploracyjne  
**Czas trwania:** 45 minut  
**Poziom:** Początkujący

---

## Cel szkoleniowy

W tym notebooku nauczysz się:
- Używać DataFrame Reader API do wczytywania danych z różnych formatów
- Konfigurować opcje czytania danych (header, delimiter, inferSchema)
- Ręcznie definiować schematy za pomocą StructType i StructField
- Wykonywać podstawowe operacje eksploracyjne na DataFrame (columns, dtypes, count, describe, summary)
- Porównywać automatyczne wykrywanie schematów z ręczną definicją

---

## Kontekst biznesowy

Organizacja KION zarządza danymi z różnych źródeł:
- **Klienci** - dane w formacie CSV
- **Zamówienia** - dane w formacie JSON
- **Produkty** - dane w formacie Parquet

Zadaniem data engineera jest:
1. Zaimportować dane z różnych formatów do Databricks
2. Zrozumieć strukturę danych i typy kolumn
3. Przygotować dane do dalszych transformacji

---

## Podstawy teoretyczne

### DataFrame Reader API

Databricks udostępnia uniwersalne API do wczytywania danych:

```python
spark.read.format("format").option("key", "value").load("path")
```

Wspierane formaty:
- **CSV** - dane tabelaryczne z separatorami
- **JSON** - dane semi-strukturalne
- **Parquet** - kolumnowy format binarny z kompresją
- **Delta** - format transakcyjny (omówiony w kolejnych modułach)

### Opcje wczytywania danych

Dla **CSV**:
- `header=True` - pierwsza linia zawiera nazwy kolumn
- `inferSchema=True` - automatyczne wykrywanie typów danych
- `delimiter=","` - separator kolumn

Dla **JSON**:
- `multiLine=True` - JSON rozłożony na wiele linii
- `inferSchema=True` - wykrywanie struktury zagnieżdżonej

### Definicja schematu

Zalecana praktyka: **ręczne definiowanie schematu** zamiast automatycznego wykrywania.

Korzyści:
- Szybsze wczytywanie (brak konieczności skanowania danych)
- Kontrola typów danych
- Uniknięcie błędów przy niepoprawnych danych

Przykład:

```python
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True)
])
```

### Operacje eksploracyjne

| Operacja | Opis |
|----------|------|
| `df.columns` | Lista nazw kolumn |
| `df.dtypes` | Lista typów kolumn |
| `df.count()` | Liczba wierszy |
| `df.printSchema()` | Struktura schematu |
| `df.describe()` | Statystyki opisowe (count, mean, stddev, min, max) |
| `df.summary()` | Rozszerzone statystyki (+ percentyle) |
| `df.show(n)` | Wyświetl n wierszy |

---

## Inicjalizacja środowiska

Wykonujemy centralny skrypt konfiguracyjny:

In [ ]:
%run ./00_setup

## Konfiguracja notebooka

Definiujemy zmienne specyficzne dla tego notebooka:

In [ ]:
# Ścieżki do katalogów z danymi (podkatalogi w DATASET_BASE_PATH z 00_setup)
CUSTOMERS_PATH = f"{DATASET_BASE_PATH}/customers"
ORDERS_PATH = f"{DATASET_BASE_PATH}/orders"
PRODUCTS_PATH = f"{DATASET_BASE_PATH}/products"

# Ścieżki do konkretnych plików
CUSTOMERS_CSV = f"{CUSTOMERS_PATH}/customers.csv"
ORDERS_JSON = f"{ORDERS_PATH}/orders_batch.json"
PRODUCTS_PARQUET = f"{PRODUCTS_PATH}/products.parquet"

print(f"Ścieżka do pliku customers CSV: {CUSTOMERS_CSV}")
print(f"Ścieżka do pliku orders JSON: {ORDERS_JSON}")
print(f"Ścieżka do pliku products Parquet: {PRODUCTS_PARQUET}")

---

## Część 1: Import danych CSV (Customers)

### 1.1. Wczytanie CSV z automatycznym wykrywaniem schematu

In [ ]:
# Wczytanie danych CSV z automatycznym wykrywaniem schematu
customers_auto_df = (
    spark.read
    .format("csv")
    .option("header", "true")       # Pierwsza linia to nazwy kolumn
    .option("inferSchema", "true")  # Automatyczne wykrywanie typów danych
    .load(CUSTOMERS_CSV)
)

# Wyświetl schemat
print("[INFO] Schemat wykryty automatycznie:")
customers_auto_df.printSchema()

# Wyświetl próbkę danych
print("\n[INFO] Próbka danych (5 wierszy):")
customers_auto_df.show(5, truncate=False)

### 1.2. Eksploracja danych CSV

In [ ]:
# Lista nazw kolumn
print("[INFO] Kolumny DataFrame:")
print(customers_auto_df.columns)

# Lista typów danych
print("\n[INFO] Typy danych kolumn:")
print(customers_auto_df.dtypes)

# Liczba wierszy
row_count = customers_auto_df.count()
print(f"\n[INFO] Liczba wierszy: {row_count}")

### 1.3. Statystyki opisowe

In [ ]:
# Statystyki opisowe (count, mean, stddev, min, max)
print("[INFO] Statystyki opisowe (describe):")
customers_auto_df.describe().show()

# Rozszerzone statystyki (+ percentyle)
print("\n[INFO] Rozszerzone statystyki (summary):")
customers_auto_df.summary().show()

### 1.4. Ręczna definicja schematu dla CSV

**Best Practice:** Definiowanie schematu manualnie zapewnia kontrolę i wydajność.

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

# Definicja schematu dla customers
# Struktura: customer_id (int), customer_name (string), email (string), country (string), registration_date (timestamp)
customers_schema = StructType([
    StructField("customer_id", IntegerType(), nullable=False),
    StructField("customer_name", StringType(), nullable=True),
    StructField("email", StringType(), nullable=True),
    StructField("country", StringType(), nullable=True),
    StructField("registration_date", TimestampType(), nullable=True)
])

# Wczytanie danych CSV z ręcznie zdefiniowanym schematem
customers_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .schema(customers_schema)  # Użycie zdefiniowanego schematu
    .load(CUSTOMERS_CSV)
)

print("[INFO] Schemat zdefiniowany ręcznie:")
customers_df.printSchema()

print("\n[INFO] Próbka danych z ręcznym schematem:")
customers_df.show(5, truncate=False)

### ✅ Ćwiczenie 1: Wczytanie CSV z własnymi opcjami

**Zadanie:** Wczytaj plik CSV `customers.csv` ponownie, ale:
- Ustaw delimiter na `;` (celowo błędny, aby zobaczyć efekt)
- Wyłącz `inferSchema`
- Zaobserwuj, jak to wpływa na wynik

**Hint:** Użyj `.option("delimiter", ";")`

In [ ]:
# TODO: Wczytaj customers.csv z delimitorem `;` i bez inferSchema
# customers_wrong_df = spark.read.format("csv")...

# TODO: Wyświetl schemat i 5 wierszy
# customers_wrong_df.printSchema()
# customers_wrong_df.show(5, truncate=False)

# Zaobserwuj, że wszystkie dane będą w jednej kolumnie
pass

---

## Część 2: Import danych JSON (Orders)

### 2.1. Wczytanie JSON z automatycznym wykrywaniem schematu

In [ ]:
# Wczytanie danych JSON z automatycznym wykrywaniem schematu
orders_auto_df = (
    spark.read
    .format("json")
    .option("inferSchema", "true")
    .load(ORDERS_JSON)
)

print("[INFO] Schemat JSON wykryty automatycznie:")
orders_auto_df.printSchema()

print("\n[INFO] Próbka danych JSON:")
orders_auto_df.show(5, truncate=False)

### 2.2. Eksploracja danych JSON

In [ ]:
# Liczba kolumn i wierszy
print(f"[INFO] Liczba kolumn: {len(orders_auto_df.columns)}")
print(f"[INFO] Nazwy kolumn: {orders_auto_df.columns}")
print(f"[INFO] Liczba wierszy: {orders_auto_df.count()}")

# Typy danych
print("\n[INFO] Typy danych:")
for col_name, col_type in orders_auto_df.dtypes:
    print(f"  - {col_name}: {col_type}")

### 2.3. Ręczna definicja schematu dla JSON

In [ ]:
from pyspark.sql.types import DoubleType

# Definicja schematu dla orders
# Struktura: order_id (int), customer_id (int), order_date (timestamp), total_amount (double), status (string)
orders_schema = StructType([
    StructField("order_id", IntegerType(), nullable=False),
    StructField("customer_id", IntegerType(), nullable=True),
    StructField("order_date", TimestampType(), nullable=True),
    StructField("total_amount", DoubleType(), nullable=True),
    StructField("status", StringType(), nullable=True)
])

# Wczytanie danych JSON z ręcznie zdefiniowanym schematem
orders_df = (
    spark.read
    .format("json")
    .schema(orders_schema)
    .load(ORDERS_JSON)
)

print("[INFO] Schemat JSON zdefiniowany ręcznie:")
orders_df.printSchema()

print("\n[INFO] Próbka danych z ręcznym schematem:")
orders_df.show(5, truncate=False)

### 2.4. Statystyki opisowe dla JSON

In [ ]:
# Statystyki dla kolumn numerycznych
print("[INFO] Statystyki dla orders:")
orders_df.select("order_id", "customer_id", "total_amount").describe().show()

# Rozkład wartości dla kolumny 'status'
print("\n[INFO] Rozkład wartości w kolumnie 'status':")
orders_df.groupBy("status").count().orderBy("count", ascending=False).show()

### ✅ Ćwiczenie 2: Eksploracja danych JSON

**Zadanie:**
1. Policz liczbę unikalnych klientów w DataFrame `orders_df`
2. Znajdź wartość maksymalną i minimalną w kolumnie `total_amount`
3. Policz liczbę zamówień w każdym statusie

**Hint:** Użyj `.select()`, `.distinct()`, `.count()`, `.agg()`, `.groupBy()`

In [ ]:
# TODO: Policz liczbę unikalnych customer_id
# unique_customers = orders_df.select("customer_id").distinct().count()
# print(f"Liczba unikalnych klientów: {unique_customers}")

# TODO: Znajdź min i max total_amount
# from pyspark.sql.functions import min, max
# orders_df.select(min("total_amount"), max("total_amount")).show()

# TODO: Policz zamówienia według statusu
# orders_df.groupBy("status").count().show()

pass

---

## Część 3: Import danych Parquet (Products)

### 3.1. Wczytanie Parquet (schemat wbudowany)

In [ ]:
# Parquet zawiera już schemat wbudowany - nie trzeba go definiować
products_df = (
    spark.read
    .format("parquet")
    .load(PRODUCTS_PARQUET)
)

print("[INFO] Schemat Parquet (wbudowany):")
products_df.printSchema()

print("\n[INFO] Próbka danych Parquet:")
products_df.show(5, truncate=False)

### 3.2. Eksploracja danych Parquet

In [ ]:
# Podstawowe informacje
print(f"[INFO] Liczba kolumn: {len(products_df.columns)}")
print(f"[INFO] Nazwy kolumn: {products_df.columns}")
print(f"[INFO] Liczba produktów: {products_df.count()}")

# Typy danych
print("\n[INFO] Typy danych:")
for col_name, col_type in products_df.dtypes:
    print(f"  - {col_name}: {col_type}")

### 3.3. Statystyki opisowe dla Parquet

In [ ]:
# Statystyki dla kolumn numerycznych
print("[INFO] Statystyki dla products:")
products_df.describe().show()

# Rozszerzone statystyki
print("\n[INFO] Rozszerzone statystyki:")
products_df.summary().show()

### 3.4. Rozkład kategorii produktów

In [ ]:
# Sprawdź, czy istnieje kolumna 'category' i wyświetl rozkład
if "category" in products_df.columns:
    print("[INFO] Rozkład kategorii produktów:")
    products_df.groupBy("category").count().orderBy("count", ascending=False).show()
else:
    print("[INFO] Kolumna 'category' nie istnieje w danych produktów")
    print(f"[INFO] Dostępne kolumny: {products_df.columns}")

### ✅ Ćwiczenie 3: Analiza danych Parquet

**Zadanie:**
1. Znajdź średnią cenę produktu (jeśli kolumna `price` istnieje)
2. Znajdź 5 najdroższych produktów
3. Policz produkty według kategorii (jeśli kolumna istnieje)

**Hint:** Użyj `.agg()`, `.orderBy()`, `.limit()`

In [ ]:
# TODO: Oblicz średnią cenę
# from pyspark.sql.functions import avg
# products_df.select(avg("price")).show()

# TODO: Znajdź 5 najdroższych produktów
# products_df.orderBy(col("price").desc()).limit(5).show(truncate=False)

# TODO: Policz produkty według kategorii
# if "category" in products_df.columns:
#     products_df.groupBy("category").count().show()

pass

---

## Część 4: Porównanie wydajności

### 4.1. Wczytanie CSV: inferSchema vs ręczny schemat

In [ ]:
import time

# Test 1: Automatyczne wykrywanie schematu
start_auto = time.time()
df_auto = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CUSTOMERS_CSV)
)
count_auto = df_auto.count()  # Action - wymusza wykonanie
time_auto = time.time() - start_auto

print(f"[BENCHMARK] Wczytanie CSV z inferSchema: {time_auto:.3f} sekund")
print(f"[BENCHMARK] Liczba wierszy: {count_auto}")

# Test 2: Ręczne zdefiniowanie schematu
start_manual = time.time()
df_manual = (
    spark.read
    .format("csv")
    .option("header", "true")
    .schema(customers_schema)
    .load(CUSTOMERS_CSV)
)
count_manual = df_manual.count()  # Action - wymusza wykonanie
time_manual = time.time() - start_manual

print(f"\n[BENCHMARK] Wczytanie CSV z ręcznym schematem: {time_manual:.3f} sekund")
print(f"[BENCHMARK] Liczba wierszy: {count_manual}")

# Porównanie
speedup = (time_auto - time_manual) / time_auto * 100
print(f"\n[WYNIK] Przyspieszenie: {speedup:.1f}%")

---

## Część 5: Best Practices

### Zalecenia przy importowaniu danych:

1. **Zawsze definiuj schemat ręcznie**
   - Szybsze wczytywanie
   - Kontrola typów danych
   - Uniknięcie błędów typu

2. **Wybieraj odpowiedni format**
   - **Parquet** - najlepszy dla analityki (kolumnowy, kompresja)
   - **CSV** - łatwy do debugowania, ale wolniejszy
   - **JSON** - elastyczny dla danych semi-strukturalnych

3. **Używaj partycjonowania**
   - Przyspieszenie zapytań filtrujących
   - Przykład: partycjonowanie zamówień według daty

4. **Sprawdzaj jakość danych od razu**
   - `count()` - sprawdź liczbę wierszy
   - `describe()` - sprawdź rozkład wartości
   - `printSchema()` - zweryfikuj typy

5. **Używaj `limit()` podczas eksperymentowania**
   - Przyspiesza iteracje nad kodem
   - Przykład: `df.limit(1000).show()`

6. **Dokumentuj schematy**
   - Ułatwia utrzymanie kodu
   - Przykład: komentarze przy definicji StructType

---

## Podsumowanie

W tym notebooku nauczyłeś się:

✅ **DataFrame Reader API**
- Wczytywanie danych z CSV, JSON, Parquet
- Konfiguracja opcji (header, inferSchema, delimiter)

✅ **Ręczna definicja schematu**
- Tworzenie schematów za pomocą StructType i StructField
- Porównanie wydajności: inferSchema vs ręczny schemat

✅ **Operacje eksploracyjne**
- Podstawowe operacje: columns, dtypes, count
- Statystyki: describe(), summary()
- Grupowanie i agregacja

✅ **Best Practices**
- Zalecenie ręcznego definiowania schematu
- Wybór formatu danych dla różnych scenariuszy
- Sprawdzanie jakości danych

---

### Następne kroki:

📌 **Kolejny notebook:** `03_basic_transformations_sql_pyspark.ipynb`  
📌 **Temat:** Transformacje danych, operacje SQL i PySpark API

---

## Zasoby dodatkowe

- [Databricks - Reading Data](https://docs.databricks.com/ingestion/index.html)
- [PySpark DataFrame API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)
- [Schema Evolution in Spark](https://docs.databricks.com/delta/schema-evolution.html)

---

## Cleanup

Zwalniamy zasoby (opcjonalne):

In [ ]:
# Odłączenie tymczasowych widoków (jeśli były tworzone)
# spark.catalog.dropTempView("customers_view")

print("[INFO] Notebook zakończony pomyślnie")